**Importação das bibliotecas**

In [70]:
import numpy as np
import scipy.linalg as sla
from scipy.linalg import lu
import time

**Funções Auxiliares**

In [71]:
def substituicao_direta(L, b):
  """
  Resolve o sistema triangular inferior Ly = b para y (Forward Substitution).
  L tem 1s na diagonal - Método Doolittle.
  """
  n = L.shape[0]
  y = np.zeros(n, dtype=float)

  for i in range(n):
    sum_j = 0
    for j in range(i):
      sum_j += L[i, j] * y[j]
      y[i] = b[i] - sum_j

    return y

def substituicao_reversa(U, y):
  """
  Resolve o sistema triangular superior Ux = y para x (Backward substitution).
  """
  n = U.shape[0]
  x = np.zeros(n, dtype=float)

  for i in range(n - 1, -1, -1):
    sum_j = 0
    for j in range(i + 1, n):
      sum_j += U[i, j] * x[j]

    if np.isclose(U[i, i], 0):
      raise ValueError("Matriz U é singular, o sistema pode não ter solução única.")

    x[i] = (y[i] - sum_j) / U[i, i]

  return x

def substituicao_retroativa(A_escalonada, b_escalonado):
  """
  Resolve um sistema linear superior triangular.
  Recebe: Matriz A (triangular superior) e vetor b.
  Retorna: Vetor solução x.
  """

  n = len(b_escalonado)
  x = np.zeros(n)

  # Percorre as linhas de baixo para cima (n-1 até 0)
  for i in range(n - 1, -1, -1):
      # Calcula a soma dos termos já conhecidos (A[i, j] * x[j])
      soma = np.dot(A_escalonada[i, i+1:], x[i+1:])

      # Calcula o valor da variável x[i]
      x[i] = (b_escalonado[i] - soma) / A_escalonada[i, i]

  return x

**Funções Principais**

In [72]:
def resolve_lu(A_in, b_in):
  """
  Implementa as três etapas do processo de decomposição LU
  """
  A = np.array(A_in, dtype=float)
  b = np.array(b_in, dtype=float)

  P, L, U = lu(A)

  Pb = P @ b
  y = substituicao_direta(L, Pb)
  x = substituicao_reversa(U, y)

  return P, L, U, y, x

def eliminacao_gaussiana_solver(A, b):
  # Cria a matriz estendida
    Ab = np.c_[np.array(A, dtype=float), np.array(b, dtype=float)]
    n = len(A)

    # Processo de ESCALONAMENTO (Eliminação)
    for c in range(n - 1): # c é o índice da coluna pivô
        # Implementação de Pivotamento Parcial (Para maior robustez - Boa prática)
        # Encontra o índice da linha com o maior valor absoluto na coluna c, de c para baixo
        p = np.abs(Ab[c:, c]).argmax() + c

        # Se o pivô não for o elemento Ab[c,c], troca as linhas c e p
        if p != c:
            Ab[[c, p]] = Ab[[p, c]]
            # Verifica se o pivô é zero (o sistema pode não ter solução única)
        if Ab[c, c] == 0:
            raise ValueError(f"O sistema não pode ser resolvido por este método: O pivô na coluna {c} é zero após o pivotamento.")

        # Faz a eliminação para as linhas l abaixo da linha c
        for l in range(c + 1, n):
            # Fator multiplicador (m)
            m = Ab[l, c] / Ab[c, c]
            # Nova linha l = Linha l - m * Linha pivô (c)
            Ab[l] = Ab[l] - m * Ab[c]

    # Extrai a nova matriz triangular superior e o vetor b
    A_escalonada = Ab[:, :-1]
    b_escalonado = Ab[:, -1]
    # Processo de SUBSTITUIÇÃO RETROATIVA
    x = substituicao_retroativa(A_escalonada, b_escalonado)

    return x

**Exercício 1 (a) A**

In [73]:
A1a = [[2, 1], [1, 2]]
b1a = [1, 2]

P, L, U, y, x = resolve_lu(A1a, b1a)
# -------- Exercício 1 (a) --------
print(f"Matriz P:\n{P}\n")
print(f"Matriz L (diagonal de 1s):\n{L}\n")
print(f"Matriz U:\n{U}\n")
print(f"Solução intermediária y (de Ly=Pb):\n{y}\n")
print(f"Solução final x (de Ux=y):\n{x}")
# Verificação:
print(f"Verificação (Ax): {np.array(A1a) @ x}")
print(f"Solução esperada: [0, 1]")

Matriz P:
[[1. 0.]
 [0. 1.]]

Matriz L (diagonal de 1s):
[[1.  0. ]
 [0.5 1. ]]

Matriz U:
[[2.  1. ]
 [0.  1.5]]

Solução intermediária y (de Ly=Pb):
[0. 0.]

Solução final x (de Ux=y):
[0. 0.]
Verificação (Ax): [0. 0.]
Solução esperada: [0, 1]


**Exercício 1 (b)**

In [74]:
A1b = [[4,1], [1,5]]
b1b = [2, 10]

P, L, U, y, x = resolve_lu(A1b, b1b)

print(f"P:\n{P}\n")
print(f"L:\n{L}\n")
print(f"U:\n{U}\n")
print(f"Solução y:\n{y}\n")
print(f"Solução x:\n{x}")
print(f"Verificação (Ax): {np.array(A1b) @ x}")
print(f"Solução esperada: [0, 2]")


P:
[[1. 0.]
 [0. 1.]]

L:
[[1.   0.  ]
 [0.25 1.  ]]

U:
[[4.   1.  ]
 [0.   4.75]]

Solução y:
[0. 0.]

Solução x:
[0. 0.]
Verificação (Ax): [0. 0.]
Solução esperada: [0, 2]


**Exercício 2 (a)**

In [75]:
A2a = [[1, 1, 1], [4, 4, 2], [2, 1, -1]]
b2a = [1, 2, 0]

# Medindo tempo da nossa função em microssegundos
start_time = time.perf_counter()
_, _, _, _, x_lu = resolve_lu(A2a, b2a)
end_time = time.perf_counter()
tempo_lu = (end_time - start_time) * 1_000_000

# Medindo tempo da função otimizada
start_time = time.perf_counter()
x_gauss = eliminacao_gaussiana_solver(A2a, b2a)
end_time = time.perf_counter()
tempo_gauss = (end_time - start_time) * 1_000_000

print(f"Solução (ambos os métodos): {x_lu}")
print(f"Tempo Decomposição LU: {tempo_lu:.2f} µs")
print(f"Tempo Eliminção Gaussiana: {tempo_gauss:.2f} µs")

Solução (ambos os métodos): [ 0. -0.  0.]
Tempo Decomposição LU: 395.99 µs
Tempo Eliminção Gaussiana: 496.33 µs


**Exercício 2 (b)**

In [76]:
A2b = [[7, -7, 1], [-3, 3, 2], [7, 7, -72]]
b2b = [1, 2, 7]

start_time = time.perf_counter()
_, _, _, _, x_lu = resolve_lu(A2b, b2b)
end_time = time.perf_counter()
tempo_lu = (end_time - start_time) * 1_000_000

start_time = time.perf_counter()
x_gauss = eliminacao_gaussiana_solver(A2b, b2b)
end_time = time.perf_counter()
tempo_gauss = (end_time - start_time) * 1_000_000

print(f"Solução (ambos os métodos): {x_lu}")
print(f"Tempo Decomposição LU: {tempo_lu:.2f} µs")
print(f"Tempo Eliminação Gaussiana: {tempo_gauss:.2f} µs")

Solução (ambos os métodos): [0. 0. 0.]
Tempo Decomposição LU: 1356.14 µs
Tempo Eliminação Gaussiana: 338.60 µs


**Exercício 2 (c)**

In [77]:
A2c = [[1, 2, 3, 4], [2, 2, 3, 4], [3, 3, 3, 4], [4, 4, 4, 4]]
b2c = [20, 22, 22, 24]

start_time = time.perf_counter()
_, _, _, _, x_lu = resolve_lu(A2c, b2c)
end_time = time.perf_counter()
tempo_lu = (end_time - start_time) * 1_000_000

start_time = time.perf_counter()
x_gauss = eliminacao_gaussiana_solver(A2c, b2c)
end_time = time.perf_counter()
tempo_gauss = (end_time - start_time) * 1_000_000

print(f"Solução (ambos os métodos): {x_lu}")
print(f"Tempo Decomposição LU: {tempo_lu:.2f} µs")
print(f"Tempo Eliminação Gaussiana: {tempo_gauss:.2f} µs")

Solução (ambos os métodos): [0. 0. 0. 0.]
Tempo Decomposição LU: 434.93 µs
Tempo Eliminação Gaussiana: 843.63 µs
